In [1]:
%run ../base_setup.ipynb

Foundation: 1.02.015
Transition: 2.03.026


In [2]:
from ds_behavioral.sample.sample_data import CallCentreSamples
from ds_behavioral.sample.sample_data import ProfileSample

In [3]:
builder = DataBuilder('synthetic_data_agent')
tools = builder.tools

### Build the Core Attributes

In [6]:
customer = Transition.from_env('synthetic_customer').load_source_canonical()
discover.data_dictionary(customer)

,Attribute,dType,%_Null,%_Dom,Count,Unique,Observations
0,age,float64,0.15,0.005,425,422,max=85.381 | min=20.468 | mean=47.12
1,balance,float64,0.00,0.004,500,496,max=992.79 | min=31.57 | mean=187.96
2,forename,object,0.00,0.004,500,499,Sample: Iva | Elinor | Kurtis
3,gender,object,0.00,0.622,500,2,Sample: M | F
4,id,object,0.00,0.002,500,500,Sample: CU_5236269 | CU_9764350 | CU_5954610
5,last_login,object,0.00,0.002,500,500,Sample: 03-29-19 17:32 | 04-20-19 09:28 | 04-18-19 10:49
6,null,float64,1.00,0.000,0,0,max=nan | min=nan | mean=nan
7,online,int64,0.00,0.788,500,2,max=1 | min=0 | mean=0.21
8,profession,object,0.10,0.242,450,15,Sample: Recruiter | Quality Control Specialist | VP Product Management
9,single cat,object,0.40,1.000,300,1,Sample: A


In [8]:
sample_size = 10000

df = pd.DataFrame()
df['call_id'] = tools.unique_identifiers(from_value=1000000, to_value=9999999, size=sample_size)

customer = Transition.from_env('synthetic_customer').load_source_canonical()

df['customer_id'] = tools.get_reference(df=customer, header='id', weight_pattern=[20,10,7,6,5,4,3,2,1,0.5], at_most=40, size=sample_size)

weekday_pattern = [5,2,4,2,1,0,0]
hour_pattern = [0,0,0,0,0,0,1,4,6,9,6,3,1,3,4,5,5,8,7,5,3,2,1,0]
df['call_date'] = tools.get_datetime(start='01/10/2018', until='31/10/2018', ordered=True, date_format='%m-%d-%Y %H:%M:%S', day_first=True,
                                   weekday_pattern=weekday_pattern, hour_pattern=hour_pattern, size=sample_size)

contact_pattern = [10,3,2,1,1,1] + [0.1]*7
contact = CallCentreSamples.contact_type(shuffle=False)
df['contact'] = tools.get_category(selection=contact, weight_pattern=contact_pattern, size=sample_size)

complaint_pattern = [10,9,7,6,5,4,3,2] + [1]*10 + [0.5]*11
complaint = CallCentreSamples.complaint(shuffle=False)
df['complaint'] = tools.get_category(selection=complaint, weight_pattern=complaint_pattern, size=sample_size)

agent_pattern = [10,5,1,0.2]
agents = ProfileSample.female_names(size=40)
df['agent'] = tools.get_category(selection=agents, weight_pattern=agent_pattern, size=sample_size)

minute_pattern = [10,5,3,1,0.5,0.2]
df['duration'] = tools.get_datetime(start='01/01/2018 00:01', until='01/01/2018 00:30', ordered=True, date_format='%M:%S', 
                                       minute_pattern=minute_pattern, size=sample_size)

df['escalated'] = tools.get_category(selection=[1,0], weight_pattern=[1,[1000,10,1000]], size=sample_size)
 
df['referred'] = tools.get_category(selection=[1,0], weight_pattern=[1,20], size=sample_size)


#### (optionally) Add volume of noise columns

In [9]:
for i in range(40):
    quantity = tools.get_number(0.005, 0.03, weight_pattern=[5,2,1,0.5])[0]
    col = "stat_{}".format(i)
    df[col] = tools.get_number(0, 1, weight_pattern=[20,1], quantity=quantity, size=sample_size)

### Save and Validate

In [11]:
filename = os.path.join(os.environ['DTU_ORIGIN_PATH'],'synthetic_agent.csv')
builder.save_to_disk(df, filename=filename)
discover.data_dictionary(df)

,Attribute,dType,%_Null,%_Dom,Count,Unique,Observations
0,agent,object,0.000,0.054,10000,40,Sample: Naomi | Nikki | Ofelia
1,call_date,object,0.000,0.000,10000,9842,Sample: 10-25-2018 18:17:39 | 10-25-2018 10:01:12 | 10-25-2018 18:31:29
2,call_id,object,0.000,0.000,10000,10000,Sample: 8385635 | 9360667 | 8615345
3,complaint,object,0.000,0.160,10000,29,Sample: Requested documentation not issued | Customer payment processed...
4,contact,object,0.000,0.532,10000,13,Sample: MyPortal | E-mail & Phone Call | Phone Call
5,customer_id,object,0.000,0.004,10000,499,Sample: CU_1137559 | CU_4095762 | CU_9949317
6,duration,object,0.000,0.003,10000,1384,Sample: 12:42 | 11:52 | 06:00
7,escalated,int64,0.000,0.954,10000,2,max=1 | min=0 | mean=0.05
8,referred,int64,0.000,0.954,10000,2,max=1 | min=0 | mean=0.05
9,stat_0,float64,0.989,0.936,110,2,max=1.0 | min=0.0 | mean=0.06
